# Creating FABnet IPv4 Network: Fully Automatic Configuration

FABRIC provides a pair of layer 3 IP networking services across every FABRIC site (FABnetv4 and FABnetv6). You can think of this service as a private internet that connects experiments across the testbed using FABRIC's high-performance network links. 

This notebook describes how to use the FABnetv4 service which is FABRIC's private IPv4 internet using FABlib's the fully automatic configuration.   


## Import the FABlib Library


In [131]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,47556e8d-33f2-4482-b0b3-30f17ecb271b
Bastion Username,rodrigo_0000140592
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


## Create the Experiment Slice

The following creates two nodes, on different sites, with basic NICs connected to FABRIC's FABnetv4 internet.  

Two nodes are created and each is automatically connected to a FABnetv4 network.  Fully automatic configuration does not require the user to manually add or configure interfaces. Instead, the `node.add_fabnet()` function is called directly on the node.  The call adds an interface and connects it to a FABnet network on the local site.  If multiple nodes reside on the same site, they will be connected to the same FABnet network.   The interface's mode is set to `auto` enabling an automatic configuration of the IP addresses during the post boot phase.

This is the simplest way to connect your nodes to FABnet and is recommended for anyone who does not need custom control over IPs and routes.


In [132]:
slice_name = 'Cassandra'
[site1,site2,site3,site4] = fablib.get_random_sites(count=4)
print(f"Sites: {site1}, {site2}, {site3}, {site4}")

node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'
node4_name = 'Node4'

network1_name='net1'
network2_name='net2'
network3_name='net3'
network4_name='net4'

node1_nic_name = 'nic1'
node2_nic_name = 'nic2'
node3_nic_name = 'nic3'
node4_nic_name = 'nic4'

Sites: CLEM, UCSD, MASS, PRIN


## Configuring Interfaces of All Nodes of Slices

In [133]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Networks
net1 = slice.add_l3network(name=network1_name, type='IPv4')
net2 = slice.add_l3network(name=network2_name, type='IPv4')
net3 = slice.add_l3network(name=network3_name, type='IPv4')
net4 = slice.add_l3network(name=network4_name, type='IPv4')


# Node1
node1 = slice.add_node(name=node1_name, site=site1, cores=4, ram=16, disk=30, image='default_ubuntu_20')
iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
iface1.set_mode('auto')
net1.add_interface(iface1)
node1.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net1.get_gateway())

# Node2
node2 = slice.add_node(name=node2_name, site=site2, cores=4, ram=16, disk=30, image='default_ubuntu_20')
iface2  = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]
iface2.set_mode('auto')
net2.add_interface(iface2)
node2.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net2.get_gateway())

# Node3
node3 = slice.add_node(name=node3_name, site=site3, cores=4, ram=16, disk=30, image='default_ubuntu_20')
iface3  = node3.add_component(model='NIC_Basic', name=node3_nic_name).get_interfaces()[0]
iface3.set_mode('auto')
net3.add_interface(iface3)
node3.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net3.get_gateway())

# Node4
node4 = slice.add_node(name=node4_name, site=site4, cores=4, ram=16, disk=30, image='default_ubuntu_20')
iface4  = node4.add_component(model='NIC_Basic', name=node4_nic_name).get_interfaces()[0]
iface4.set_mode('auto')
net4.add_interface(iface4)
node4.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net4.get_gateway())

#Submit Slice Request
slice_id = slice.submit();


Retry: 8, Time: 280 sec


ID,89500f5b-1a59-4c20-aafe-2c7d2d5d7b75
Name,Cassandra
Lease Expiration (UTC),2023-12-06 00:17:30 +0000
Lease Start (UTC),2023-12-05 00:17:31 +0000
Project ID,47556e8d-33f2-4482-b0b3-30f17ecb271b
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
54f02621-3e62-455f-b61e-f504863444fa,Node1,4,16,100,default_ubuntu_20,qcow2,clem-w1.fabric-testbed.net,CLEM,ubuntu,2620:103:a006:12:f816:3eff:fe41:525a,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fe41:525a,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
2427b880-65e2-4163-8ee5-27f21af853e6,Node2,4,16,100,default_ubuntu_20,qcow2,ucsd-w3.fabric-testbed.net,UCSD,ubuntu,132.249.252.189,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.189,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
75ce08f5-84ae-47f1-a7de-dddca5b73b83,Node3,4,16,100,default_ubuntu_20,qcow2,mass-w3.fabric-testbed.net,MASS,ubuntu,205.172.170.125,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@205.172.170.125,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1fe35c5f-cc63-4fc5-8d45-3066f22ca7e1,Node4,4,16,100,default_ubuntu_20,qcow2,prin-w2.fabric-testbed.net,PRIN,ubuntu,2620:c4:0:180:f816:3eff:fec6:92fa,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:c4:0:180:f816:3eff:fec6:92fa,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
3435a8d7-19e7-4f29-b643-b9eeeda291a4,net1,L3,FABNetv4,CLEM,10.136.3.0/24,10.136.3.1,Active,
305cc932-a186-4b78-8d9d-58af28eba603,net2,L3,FABNetv4,UCSD,10.134.133.0/24,10.134.133.1,Active,
853ffa2d-87e4-4461-850f-97cb7b1a8d90,net3,L3,FABNetv4,MASS,10.131.131.0/24,10.131.131.1,Active,
d530024a-e3d2-4d47-9bf1-7d9cf497e0f1,net4,L3,FABNetv4,PRIN,10.139.131.0/24,10.139.131.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
Node1-nic1-p1,p1,Node1,net1,100,auto,,06:F7:67:38:E7:26,enp6s0,enp6s0,10.136.3.2,6
Node2-nic2-p1,p1,Node2,net2,100,auto,,06:AE:3B:35:9F:83,enp6s0,enp6s0,10.134.133.2,4
Node3-nic3-p1,p1,Node3,net3,100,auto,,16:8A:F2:B1:13:F8,enp7s0,enp7s0,10.131.131.2,4
Node4-nic4-p1,p1,Node4,net4,100,auto,,02:F8:A5:DB:47:5C,enp7s0,enp7s0,10.139.131.2,4



Time to print interfaces 280 seconds


## Run the Experiment - Ping test

With automatic configuration the slice is ready for experimentation after it becomes active.  Note that automatic configuration works well when saving slices to a file and reinstantiating the slice.  Configuration tasks can be stored in the saved slice, reducing the complexity of notebooks and other runtime steps.

We will find the ping round trip time for this pair of sites.  Your experiment should be more interesting!


In [134]:
node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name) 
node3 = slice.get_node(name=node3_name)        
node4 = slice.get_node(name=node4_name) 

node1_addr = node1.get_interface(network_name=network1_name).get_ip_addr()
node2_addr = node2.get_interface(network_name=network2_name).get_ip_addr()
node3_addr = node3.get_interface(network_name=network3_name).get_ip_addr()
node4_addr = node4.get_interface(network_name=network4_name).get_ip_addr()

node1_net = node1.get_interface(network_name=network1_name).network.get_subnet()
node2_net = node2.get_interface(network_name=network2_name).network.get_subnet()
node3_net = node3.get_interface(network_name=network3_name).network.get_subnet()
node4_net = node4.get_interface(network_name=network4_name).network.get_subnet()

stdout, stderr = node1.execute(f'ping -c 1 {node2_addr}')
stdout, stderr = node1.execute(f'ping -c 1 {node3_addr}')
stdout, stderr = node1.execute(f'ping -c 1 {node4_addr}')

node1_if_name = node1.get_interface(network_name=network1_name).get_device_name()
node2_if_name = node2.get_interface(network_name=network2_name).get_device_name()
node3_if_name = node3.get_interface(network_name=network3_name).get_device_name()
node4_if_name = node4.get_interface(network_name=network4_name).get_device_name()

print("IP Addr Node 1: "+str(node1_addr))
print("IP Addr Node 2: "+str(node2_addr))
print("IP Addr Node 3: "+str(node3_addr))
print("IP Addr Node 4: "+str(node4_addr))

print("IF Name Node 1: "+str(node1_if_name))
print("IF Name Node 2: "+str(node2_if_name))
print("IF Name Node 3: "+str(node3_if_name))
print("IF Name Node 4: "+str(node4_if_name))


PING 10.134.133.2 (10.134.133.2) 56(84) bytes of data.
64 bytes from 10.134.133.2: icmp_seq=1 ttl=58 time=81.1 ms

--- 10.134.133.2 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 81.059/81.059/81.059/0.000 ms
PING 10.131.131.2 (10.131.131.2) 56(84) bytes of data.
64 bytes from 10.131.131.2: icmp_seq=1 ttl=61 time=31.3 ms

--- 10.131.131.2 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 31.297/31.297/31.297/0.000 ms
PING 10.139.131.2 (10.139.131.2) 56(84) bytes of data.
64 bytes from 10.139.131.2: icmp_seq=1 ttl=60 time=23.7 ms

--- 10.139.131.2 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 23.715/23.715/23.715/0.000 ms
IP Addr Node 1: 10.136.3.2
IP Addr Node 2: 10.134.133.2
IP Addr Node 3: 10.131.131.2
IP Addr Node 4: 10.139.131.2
IF Name Node 1: enp6s0
IF Name Node 2: enp6s0
IF Name Node 3: enp7s0
IF Name Node 4: enp7s0

## Configuring Default Routes to avoid 10.96.0.1:443 network unreachable

In [135]:


try:
    
    #Configuring Default route in nodes
    stdout, stderr = node1.execute(f'sudo ip route add default via {node1_addr} dev {node1_if_name} proto dhcp src {node1_addr} metric 100')
    print (stdout)
    
    stdout, stderr = node2.execute(f'sudo ip route add default via {node2_addr} dev {node2_if_name} proto dhcp src {node2_addr} metric 100')
    print (stdout)
    
    stdout, stderr = node3.execute(f'sudo ip route add default via {node3_addr} dev {node3_if_name} proto dhcp src {node3_addr} metric 100')
    print (stdout)
    
    stdout, stderr = node4.execute(f'sudo ip route add default via {node4_addr} dev {node4_if_name} proto dhcp src {node4_addr} metric 100')
    print (stdout)
    
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()



 RTNETLINK answers: File exists
 
 RTNETLINK answers: File exists
 



## Inserting values into Hosts file of each machine

In [136]:
# Dicionário com os objetos de nós e seus endereços IP correspondentes
nodes = {
    node1: node1_addr,
    node2: node2_addr,
    node3: node3_addr,
    node4: node4_addr
}

# Comando para adicionar as informações no /etc/hosts de cada nó
for node, address in nodes.items():
    # Criar uma lista de entradas para todos os outros nós
    other_nodes = [f"{addr} {name.get_name()}" for name, addr in nodes.items() if name != node]

    # Adicionar a entrada do próprio nó no início da lista de outras entradas
    own_node_entry = f"{address} {node.get_name()}"
    all_entries = [own_node_entry] + other_nodes

    # Combinar todas as entradas em uma única string, separando por quebras de linha
    entries = '\n'.join(all_entries)

    # Comando para adicionar as informações no /etc/hosts do nó atual
    command = f"sudo sh -c 'echo \"{entries}\" > /etc/hosts'"
    
    # Executar o comando no nó atual
    node.execute(command)

## Configuring Kubernetes

### Master Control Plane

In [137]:
try:
    file_attributes = node1.upload_file(local_file_path="/home/fabric/work/kubernetes_files/config_control_plane.sh", remote_file_path="config_control_plane.sh")

    stdout, stderr = node1.execute(f"chmod +x config_control_plane.sh && ./config_control_plane.sh")

except Exception as e:
    print(f"Exception: {e}")
 



Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2604 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [402 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [13.2 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2411 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/restricted Translation-en [336 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 c-n-f Metadata [552 B]
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [914 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/universe Translation-en [192 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/uni

### Starting Control Plane - Master _Node1_

In [138]:
import re
key_string = ''
try:
    file_attributes = node1.upload_file(local_file_path="/home/fabric/work/kubernetes_files/start_control_plane.sh", remote_file_path="start_control_plane.sh")
    
    key_string, stderr = node1.execute(f"chmod +x start_control_plane.sh && ./start_control_plane.sh {node1_net} {node1_addr}")
except Exception as e:
    print(f"Exception: {e}")

W1205 00:23:29.222164    6838 preflight.go:56] [reset] WARNING: Changes made to this host by 'kubeadm init' or 'kubeadm join' will be reverted.
[reset] Are you sure you want to proceed? [y/N]: [preflight] Running pre-flight checks
W1205 00:23:29.222212    6838 removeetcdmember.go:106] [reset] No kubeadm config, using etcd pod spec to get data directory
[reset] Deleted contents of the etcd data directory: /var/lib/etcd
[reset] Stopping the kubelet service
[reset] Unmounting mounted directories in "/var/lib/kubelet"
W1205 00:23:29.225943    6838 cleanupnode.go:134] [reset] Failed to evaluate the "/var/lib/kubelet" directory. Skipping its unmount and cleanup: lstat /var/lib/kubelet: no such file or directory
[reset] Deleting contents of directories: [/etc/kubernetes/manifests /etc/kubernetes/pki]
[reset] Deleting files: [/etc/kubernetes/admin.conf /etc/kubernetes/kubelet.conf /etc/kubernetes/bootstrap-kubelet.conf /etc/kubernetes/controller-manager.conf /etc/kubernetes/scheduler.conf]

Th

In [139]:
# Expressões regulares para encontrar os valores após "--token" e "--discovery-token-ca-cert-hash"
padrao_token = r"--token\s+([\w\.]+)"
padrao_cert_hash = r"--discovery-token-ca-cert-hash\s+(\S+)"

# Encontrar os valores usando expressões regulares
token_encontrado = re.search(padrao_token, key_string)
cert_hash_encontrado = re.search(padrao_cert_hash, key_string)

# Verificar se os valores foram encontrados e atribuí-los às variáveis correspondentes
if token_encontrado:
    token = token_encontrado.group(1)
else:
    token = None

if cert_hash_encontrado:
    cert_hash = cert_hash_encontrado.group(1)
else:
    cert_hash = None

# Imprimir os valores encontrados
print("Token:", token)
print("Cert Hash:", cert_hash)

Token: wdscpq.22w17241icroi7n2
Cert Hash: sha256:384631acaaf76c0554dd0f3c48c6b5cd111e9f14c7efbe0df6c462cf28b07b1e


### Configuring workers

In [140]:
try:
    
    file_attributes = node2.upload_file(local_file_path="/home/fabric/work/kubernetes_files/config_worker_node.sh", remote_file_path="config_worker_node.sh")
    file_attributes = node3.upload_file(local_file_path="/home/fabric/work/kubernetes_files/config_worker_node.sh", remote_file_path="config_worker_node.sh")
    file_attributes = node4.upload_file(local_file_path="/home/fabric/work/kubernetes_files/config_worker_node.sh", remote_file_path="config_worker_node.sh")
    
    stdout, stderr = node2.execute(f"chmod +x config_worker_node.sh && ./config_worker_node.sh")
    stdout, stderr = node3.execute(f"chmod +x config_worker_node.sh && ./config_worker_node.sh")
    stdout, stderr = node4.execute(f"chmod +x config_worker_node.sh && ./config_worker_node.sh")
    
except Exception as e:
    print(f"Exception: {e}")



Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2604 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [402 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [13.2 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2411 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted Translation-en [336 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 c-n-f Metadata [552 B]
Get:12 http://security.ubuntu.com/ubuntu focal-security/un

## Starting Workers Nodes

In [141]:
try:
    file_attributes = node2.upload_file(local_file_path="/home/fabric/work/kubernetes_files/start_worker_node.sh", remote_file_path="start_worker_node.sh")
    file_attributes = node3.upload_file(local_file_path="/home/fabric/work/kubernetes_files/start_worker_node.sh", remote_file_path="start_worker_node.sh")
    file_attributes = node4.upload_file(local_file_path="/home/fabric/work/kubernetes_files/start_worker_node.sh", remote_file_path="start_worker_node.sh")
    
    stdout, stderr = node2.execute(f"chmod +x start_worker_node.sh && ./start_worker_node.sh {node1_addr} {token} {cert_hash}")
    stdout, stderr = node3.execute(f"chmod +x start_worker_node.sh && ./start_worker_node.sh {node1_addr} {token} {cert_hash}")
    stdout, stderr = node4.execute(f"chmod +x start_worker_node.sh && ./start_worker_node.sh {node1_addr} {token} {cert_hash}")
except Exception as e:
    print(f"Exception: {e}")

10.136.3.2
W1205 00:26:22.415383    6876 preflight.go:56] [reset] WARNING: Changes made to this host by 'kubeadm init' or 'kubeadm join' will be reverted.
[reset] Are you sure you want to proceed? [y/N]: [preflight] Running pre-flight checks
W1205 00:26:22.415456    6876 removeetcdmember.go:106] [reset] No kubeadm config, using etcd pod spec to get data directory
[reset] Deleted contents of the etcd data directory: /var/lib/etcd
[reset] Stopping the kubelet service
[reset] Unmounting mounted directories in "/var/lib/kubelet"
W1205 00:26:22.421876    6876 cleanupnode.go:134] [reset] Failed to evaluate the "/var/lib/kubelet" directory. Skipping its unmount and cleanup: lstat /var/lib/kubelet: no such file or directory
[reset] Deleting contents of directories: [/etc/kubernetes/manifests /etc/kubernetes/pki]
[reset] Deleting files: [/etc/kubernetes/admin.conf /etc/kubernetes/kubelet.conf /etc/kubernetes/bootstrap-kubelet.conf /etc/kubernetes/controller-manager.conf /etc/kubernetes/schedule

## Getting Cluster Information

In [142]:
try:
    stdout, stderr = node1.execute("kubectl get pods --all-namespaces")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

NAMESPACE          NAME                                       READY   STATUS              RESTARTS   AGE
calico-apiserver   calico-apiserver-dbb4b494b-8c926           1/1     Running             0          109s
calico-apiserver   calico-apiserver-dbb4b494b-frmtx           1/1     Running             0          109s
calico-system      calico-kube-controllers-6c655fbc6f-m9mww   1/1     Running             0          2m23s
calico-system      calico-node-6qww9                          0/1     Init:0/2            0          2s
calico-system      calico-node-bgppt                          1/1     Running             0          2m23s
calico-system      calico-node-cjtpt                          0/1     Init:1/2            0          5s
calico-system      calico-node-wr4vm                          0/1     Init:1/2            0          10s
calico-system      calico-typha-f85b6b49c-mlmps               1/1     Running             0          2m24s
calico-system      calico-typha-f85b6b49c-zs82l  

# Cassandra Cluster

In [143]:
try:
    stdout, stderr = node1.execute("git clone https://github.com/romoreira/SFI2-Distributed-Orchestration.git")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
    stdout, stderr = node1.execute("kubectl apply -f /home/ubuntu/SFI2-Distributed-Orchestration/3rd/svc-cassandra.yaml ; kubectl apply -f /home/ubuntu/SFI2-Distributed-Orchestration/3rd/sc.yaml ; kubectl apply -f /home/ubuntu/SFI2-Distributed-Orchestration/3rd/pv-cassandra.yaml ; kubectl apply -f /home/ubuntu/SFI2-Distributed-Orchestration/3rd/pvc-cassandra.yaml ; kubectl apply -f /home/ubuntu/SFI2-Distributed-Orchestration/3rd/sts-cassandra.yaml")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")
 

 Cloning into 'SFI2-Distributed-Orchestration'...
 stdout: 
stderr: Cloning into 'SFI2-Distributed-Orchestration'...

service/cassandra created
storageclass.storage.k8s.io/standard created
persistentvolume/cassandra-pv created
persistentvolumeclaim/cassandra-pvc created
statefulset.apps/cassandra created
stdout: service/cassandra created
storageclass.storage.k8s.io/standard created
persistentvolume/cassandra-pv created
persistentvolumeclaim/cassandra-pvc created
statefulset.apps/cassandra created

stderr: 


## See Running Pods

In [144]:
try:
    stdout, stderr = node1.execute("kubectl get pods")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

NAME          READY   STATUS              RESTARTS   AGE
cassandra-0   0/1     ContainerCreating   0          1s
stdout: NAME          READY   STATUS              RESTARTS   AGE
cassandra-0   0/1     ContainerCreating   0          1s

stderr: 


## See Cassandra Cluster

In [145]:
try:
    stdout, stderr = node1.execute("kubectl logs cassandra-0")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

 Error from server (BadRequest): container "cassandra" in pod "cassandra-0" is waiting to start: ContainerCreating
 stdout: 
stderr: Error from server (BadRequest): container "cassandra" in pod "cassandra-0" is waiting to start: ContainerCreating



## See Running Pods - Cassandra

In [146]:
try:
    stdout, stderr = node1.execute("kubectl exec -it cassandra-0 -- nodetool status")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")
 

 Unable to use a TTY - input is not a terminal or the right kind of file
error: unable to upgrade connection: container not found ("cassandra")
 stdout: 
stderr: Unable to use a TTY - input is not a terminal or the right kind of file
error: unable to upgrade connection: container not found ("cassandra")



## Installing K-Sniff to Collect Packets of Cassandra Pods

In [150]:
try:
    stdout, stderr = node1.execute("sudo apt-get -y install libpcap-dev")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

 error: unknown command "sniff" for "kubectl"

Did you mean this?
	diff
 stdout: 
stderr: error: unknown command "sniff" for "kubectl"

Did you mean this?
	diff



## Preparing to Krew

In [176]:
try:
    file_attributes = node1.upload_file(local_file_path="/home/fabric/work/kubernetes_files/krew_installation.sh", remote_file_path="krew_installation.sh")
    
    stdout, stderr = node1.execute(f"chmod +x krew_installation.sh && ./krew_installation.sh")

except Exception as e:
    print(f"Exception: {e}")

./LICENSE
./krew-linux_amd64
 +++ mktemp -d
++ cd /tmp/tmp.cM1uMi8pwS
+++ uname
+++ tr '[:upper:]' '[:lower:]'
++ OS=linux
+++ uname -m
+++ sed -e s/x86_64/amd64/ -e 's/\(arm\)\(64\)\?.*/\1\2/' -e 's/aarch64$/arm64/'
++ ARCH=amd64
++ KREW=krew-linux_amd64
++ curl -fsSLO https://github.com/kubernetes-sigs/krew/releases/latest/download/krew-linux_amd64.tar.gz
++ tar zxvf krew-linux_amd64.tar.gz
  ++ ./krew-linux_amd64 install krew
the following to your ~/.bash_profile or ~/.bashrc:

    export PATH="${KREW_ROOT:-$HOME/.krew}/bin:$PATH"

and restart your shell.

  Updated the local copy of plugin index.
  Installing plugin: krew
W1205 00:50:04.246264   34083 install.go:160] Skipping plugin "krew", it is already installed
  Updated the local copy of plugin index.
  Installing plugin: sniff
  W1205 00:50:04.705072   34100 install.go:160] Skipping plugin "sniff", it is already installed
 


## Delete the Slice

Please delete your slice when you are done with your experiment.

In [124]:
try:
    slice = fablib.get_slice(slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")